# Project: Perception on Public Security in the United States of Mexico

## Introduction

After a recent trip to Mexico, I decided to explore the website of the National Institute of Statistics and Geography of Mexico ([INEGI](https://en.www.inegi.org.mx/default.html)) and use this project as an opportunity to learn something more about the country. 

After exploring the data available on the website, I decided to investigate the issue of security, particalarly perceived security, within the country. I chose this topic for 2 main reasons: 

1) While planning my trip, I was surprised of the concern expressed by my Mexican friends (both male and female). Also, during my trip, I noticed that my own perception of security (or lack thereof) was different, based on the city/town I was visiting. 

2) The data available on the INEGI website looks complex, and interesting enough to investigate: the survey has been iterated for a number of years (yearly, from 2010 to 2025), and it's divided by state, which allows both a temporal and a geographic analysis. 

### Project scope

The goal of this project is to investigate the following points: 

- How does perception of public security in Mexico change, based on the state? 

- Has it changed, overall, in the past 15 years? 

- What are the main concerns that influence perceived security (or lack of it)? 

## 1. Sourcing the data 

## 2. Cleaning up the data 

## 3. Analysis  

## 4. Conclusion

## End